In [6]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
import numpy as np
import time
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from datetime import datetime
from torchvision.models import resnet18, ResNet18_Weights


In [ ]:
# 指定 GPU 索引（假设选择 GPU 0）
GPU_INDEX = 0

# 检查 GPU 是否可用并设置设备
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please enable a GPU.")

device = torch.device(f"cuda:{GPU_INDEX}")
torch.cuda.set_device(GPU_INDEX)
print(f"Using device: {device}")

# 数据加载
class ImageDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

def load_image_paths(folder, plot_type='scatter_plots', limit_per_class=1000):
    file_paths = []
    labels = []
    label = 0  # 为每个发射机分配标签
    class_count = {}

    for tx_id in os.listdir(folder):
        tx_folder = os.path.join(folder, tx_id)
        if os.path.isdir(tx_folder):
            plot_folder = os.path.join(tx_folder, plot_type)
            if os.path.exists(plot_folder):
                tx_files = [f for f in os.listdir(plot_folder) if f.endswith('.png')]
                
                # 限制每个发射机类的图片数量
                tx_files = tx_files[:limit_per_class]

                for filename in tx_files:
                    file_paths.append(os.path.join(plot_folder, filename))
                    labels.append(label)

                # 统计每个类的图片数量
                class_count[label] = len(tx_files)

            label += 1

    # 打印每个类的图片数量
    print(f"Total classes: {len(class_count)}")
    for lbl, count in class_count.items():
        print(f"Class {lbl}: {count} images")

    return file_paths, labels

# 模型构建
def build_resnet_model(num_classes):
    # 使用预训练的ResNet模型
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)  # 加载预训练权重
    # 替换最后一层
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    return model

# 训练和评估
def train_and_evaluate_model(model, train_loader, val_loader, test_loader, device, epochs=20, lr=0.001, patience=5):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model = model.to(device)
    best_val_acc = 0
    best_epoch = 0
    train_losses, val_losses = [], []
    early_stop_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct, total = 0, 0

        # Training loop
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total
        train_losses.append(train_loss / len(train_loader))

        # Validation loop
        model.eval()
        val_loss = 0
        correct, total = 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total
        val_losses.append(val_loss / len(val_loader))

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_acc:.4f}")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = model.state_dict()
            best_epoch = epoch
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print("Early stopping triggered!")
                break

    print(f"Best Validation Accuracy: {best_val_acc:.4f} at epoch {best_epoch+1}")
    model.load_state_dict(best_model)

    # Test loop
    model.eval()
    correct, total = 0, 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = correct / total
    print(f"Test Accuracy: {test_acc:.4f}")

    # Generate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    cm_fig = plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=list(range(len(set(all_labels)))), yticklabels=list(range(len(set(all_labels)))))
    cm_fig.savefig("confusion_matrix.png")
    
    return model, train_losses, val_losses, test_acc, cm


# 比较训练效果并保存结果
def compare_scatter_trajectory_training(folder, batch_size=32, epochs=20, lr=0.001):
    print("Loading scatter plot images...")
    scatter_paths, scatter_labels = load_image_paths(folder, plot_type='scatter_plots', limit_per_class=1000)
    print("Loading trajectory plot images...")
    trajectory_paths, trajectory_labels = load_image_paths(folder, plot_type='trajectory_plots', limit_per_class=1000)

    # 数据集分割
    scatter_train_paths, scatter_test_paths, scatter_train_labels, scatter_test_labels = train_test_split(
        scatter_paths, scatter_labels, test_size=0.2, random_state=42
    )
    trajectory_train_paths, trajectory_test_paths, trajectory_train_labels, trajectory_test_labels = train_test_split(
        trajectory_paths, trajectory_labels, test_size=0.2, random_state=42
    )

    # 数据预处理
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # 构造数据集和数据加载器
    scatter_train_dataset = ImageDataset(scatter_train_paths, scatter_train_labels, transform)
    scatter_test_dataset = ImageDataset(scatter_test_paths, scatter_test_labels, transform)
    trajectory_train_dataset = ImageDataset(trajectory_train_paths, trajectory_train_labels, transform)
    trajectory_test_dataset = ImageDataset(trajectory_test_paths, trajectory_test_labels, transform)

    scatter_train_loader = DataLoader(scatter_train_dataset, batch_size=batch_size, shuffle=True)
    scatter_test_loader = DataLoader(scatter_test_dataset, batch_size=batch_size, shuffle=False)
    trajectory_train_loader = DataLoader(trajectory_train_dataset, batch_size=batch_size, shuffle=True)
    trajectory_test_loader = DataLoader(trajectory_test_dataset, batch_size=batch_size, shuffle=False)

    # 获取类别数量
    num_classes = len(set(scatter_labels))

    # 当前时间戳
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    folder_name = f"{timestamp}-1000_images_per_class-{num_classes}_classes"
    os.makedirs(folder_name, exist_ok=True)

    # 保存训练的模型和混淆矩阵
    # 构建和训练散点图模型
    print("\nTraining scatter plot model...")
    scatter_model = build_resnet_model(num_classes)
    scatter_model, scatter_train_losses, scatter_val_losses, scatter_test_acc, scatter_cm = train_and_evaluate_model(
        scatter_model, scatter_train_loader, scatter_test_loader, scatter_test_loader, device, epochs, lr
    )
    
    torch.save(scatter_model.state_dict(), os.path.join(folder_name, 'scatter_model.pth'))
    plt.savefig(os.path.join(folder_name, 'scatter_confusion_matrix.png'))

    # 保存训练日志
    with open(os.path.join(folder_name, 'scatter_training_log.txt'), 'w') as f:
        f.write(f"Training Results - Scatter Plot\n")
        f.write(f"Test Accuracy: {scatter_test_acc:.4f}\n")
        f.write(f"Train Losses: {scatter_train_losses}\n")
        f.write(f"Val Losses: {scatter_val_losses}\n")

    # 构建和训练轨迹图模型
    print("\nTraining trajectory plot model...")
    trajectory_model = build_resnet_model(num_classes)
    trajectory_model, trajectory_train_losses, trajectory_val_losses, trajectory_test_acc, trajectory_cm = train_and_evaluate_model(
        trajectory_model, trajectory_train_loader, trajectory_test_loader, trajectory_test_loader, device, epochs, lr
    )
    
    torch.save(trajectory_model.state_dict(), os.path.join(folder_name, 'trajectory_model.pth'))
    plt.savefig(os.path.join(folder_name, 'trajectory_confusion_matrix.png'))

    # 保存训练日志
    with open(os.path.join(folder_name, 'trajectory_training_log.txt'), 'w') as f:
        f.write(f"Training Results - Trajectory Plot\n")
        f.write(f"Test Accuracy: {trajectory_test_acc:.4f}\n")
        f.write(f"Train Losses: {trajectory_train_losses}\n")
        f.write(f"Val Losses: {trajectory_val_losses}\n")

# 使用示例：比较散点图和轨迹图的训练效果
folder = "../../IQ_signal_plots"  # 图像存储的文件夹
compare_scatter_trajectory_training(folder, batch_size=32, epochs=20, lr=0.001)


Using device: cuda:0
Loading scatter plot images...
Total classes: 150
Class 0: 1000 images
Class 1: 1000 images
Class 2: 1000 images
Class 3: 1000 images
Class 4: 1000 images
Class 5: 1000 images
Class 6: 1000 images
Class 7: 1000 images
Class 8: 1000 images
Class 9: 1000 images
Class 10: 1000 images
Class 11: 1000 images
Class 12: 1000 images
Class 13: 1000 images
Class 14: 1000 images
Class 15: 1000 images
Class 16: 1000 images
Class 17: 1000 images
Class 18: 1000 images
Class 19: 1000 images
Class 20: 1000 images
Class 21: 1000 images
Class 22: 1000 images
Class 23: 1000 images
Class 24: 1000 images
Class 25: 1000 images
Class 26: 1000 images
Class 27: 1000 images
Class 28: 1000 images
Class 29: 1000 images
Class 30: 1000 images
Class 31: 1000 images
Class 32: 1000 images
Class 33: 1000 images
Class 34: 1000 images
Class 35: 1000 images
Class 36: 1000 images
Class 37: 1000 images
Class 38: 1000 images
Class 39: 1000 images
Class 40: 1000 images
Class 41: 1000 images
Class 42: 1000 

Epoch 1/20 - Training:   3%|▎         | 100/3750 [00:19<12:02,  5.05it/s]

In [2]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from PIL import Image
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 指定 GPU 索引（假设选择 GPU 0）
GPU_INDEX = 1

# 检查 GPU 是否可用并设置设备
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please enable a GPU.")

device = torch.device(f"cuda:{GPU_INDEX}")
torch.cuda.set_device(GPU_INDEX)
print(f"Using device: {device}")

# 数据加载
class ImageDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

def load_image_paths(folder, plot_type='scatter_plots', limit_per_class=1000):
    file_paths = []
    labels = []
    label = 0  # 为每个发射机分配标签
    class_count = {}

    for tx_id in os.listdir(folder):
        tx_folder = os.path.join(folder, tx_id)
        if os.path.isdir(tx_folder):
            plot_folder = os.path.join(tx_folder, plot_type)
            if os.path.exists(plot_folder):
                tx_files = [f for f in os.listdir(plot_folder) if f.endswith('.png')]
                
                # 随机抽取每类数据集中的图片
                tx_files = np.random.choice(tx_files, min(limit_per_class, len(tx_files)), replace=False)

                for filename in tx_files:
                    file_paths.append(os.path.join(plot_folder, filename))
                    labels.append(label)

                # 统计每个类的图片数量
                class_count[label] = len(tx_files)

            label += 1

    # 打印每个类的图片数量
    print(f"Total classes: {len(class_count)}")
    for lbl, count in class_count.items():
        print(f"Class {lbl}: {count} images")

    return file_paths, labels, class_count

# 模型构建
def build_resnet_model(num_classes):
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    # 替换最后一层
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    return model

# 训练和评估
def train_and_evaluate_model(model, train_loader, val_loader, test_loader, device, epochs=20, lr=0.001, early_stopping_patience=5):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model = model.to(device)
    best_val_acc = 0
    best_model = None
    train_losses, val_losses = [], []
    early_stop_counter = 0
    results = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct, total = 0, 0

        # Training loop
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total
        train_losses.append(train_loss / len(train_loader))

        # Validation loop
        model.eval()
        val_loss = 0
        correct, total = 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total
        val_losses.append(val_loss / len(val_loader))

        # 输出当前训练信息
        results.append(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_acc:.4f}, "
                       f"Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_acc:.4f}")

        # 早停机制
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = model.state_dict()
            early_stop_counter = 0  # reset counter if validation accuracy improves
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # 加载最佳模型
    if best_model is not None:
        model.load_state_dict(best_model)

    # Test loop
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = correct / total
    results.append(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")

    return model, results, test_acc

# 绘制混淆矩阵
def plot_confusion_matrix(model, test_loader, device, class_names, save_path):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig(save_path)
    plt.close()

# 比较训练效果
def compare_scatter_trajectory_training(folder, batch_size=32, epochs=20, lr=0.001, limit_per_class=1000):
    print("Loading scatter plot images...")
    scatter_paths, scatter_labels, scatter_class_count = load_image_paths(folder, plot_type='scatter_plots', limit_per_class=limit_per_class)
    print("Loading trajectory plot images...")
    trajectory_paths, trajectory_labels, trajectory_class_count = load_image_paths(folder, plot_type='trajectory_plots', limit_per_class=limit_per_class)

    # 数据集分割
    scatter_train_paths, scatter_test_paths, scatter_train_labels, scatter_test_labels = train_test_split(
        scatter_paths, scatter_labels, test_size=0.2, random_state=42
    )
    trajectory_train_paths, trajectory_test_paths, trajectory_train_labels, trajectory_test_labels = train_test_split(
        trajectory_paths, trajectory_labels, test_size=0.2, random_state=42
    )

    # 数据预处理
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # 构造数据集和数据加载器
    scatter_train_dataset = ImageDataset(scatter_train_paths, scatter_train_labels, transform)
    scatter_test_dataset = ImageDataset(scatter_test_paths, scatter_test_labels, transform)
    trajectory_train_dataset = ImageDataset(trajectory_train_paths, trajectory_train_labels, transform)
    trajectory_test_dataset = ImageDataset(trajectory_test_paths, trajectory_test_labels, transform)

    scatter_train_loader = DataLoader(scatter_train_dataset, batch_size=batch_size, shuffle=True)
    scatter_test_loader = DataLoader(scatter_test_dataset, batch_size=batch_size, shuffle=False)
    trajectory_train_loader = DataLoader(trajectory_train_dataset, batch_size=batch_size, shuffle=True)
    trajectory_test_loader = DataLoader(trajectory_test_dataset, batch_size=batch_size, shuffle=False)

    # 获取类别数量
    num_classes = len(set(scatter_labels))

    # 获取每类图片数量
    scatter_class_sizes = list(scatter_class_count.values())
    trajectory_class_sizes = list(trajectory_class_count.values())

    # 获取当前时间戳，用于创建文件夹
    timestamp = time.strftime("%Y%m%d-%H%M%S", time.localtime())
    
    # 创建文件夹
    output_dir = f"{timestamp}-{len(scatter_class_sizes)}classes"
    os.makedirs(output_dir, exist_ok=True)

    # 创建和训练散点图模型
    print("\nTraining scatter plot model...")
    scatter_model = build_resnet_model(num_classes)
    scatter_model, scatter_results, scatter_test_acc = train_and_evaluate_model(
        scatter_model, scatter_train_loader, scatter_test_loader, scatter_test_loader, device, epochs, lr
    )

    # 绘制并保存混淆矩阵
    plot_confusion_matrix(scatter_model, scatter_test_loader, device, scatter_class_sizes, os.path.join(output_dir, 'scatter_confusion_matrix.png'))

    # 保存训练结果
    with open(os.path.join(output_dir, 'scatter_training_results.txt'), 'w') as f:
        for line in scatter_results:
            f.write(line + '\n')

    # 保存模型
    torch.save(scatter_model.state_dict(), os.path.join(output_dir, 'scatter_best_model.pth'))

    # 构建并训练轨迹图模型
    print("\nTraining trajectory plot model...")
    trajectory_model = build_resnet_model(num_classes)
    trajectory_model, trajectory_results, trajectory_test_acc = train_and_evaluate_model(
        trajectory_model, trajectory_train_loader, trajectory_test_loader, trajectory_test_loader, device, epochs, lr
    )

    # 绘制并保存混淆矩阵
    plot_confusion_matrix(trajectory_model, trajectory_test_loader, device, trajectory_class_sizes, os.path.join(output_dir, 'trajectory_confusion_matrix.png'))

    # 保存训练结果
    with open(os.path.join(output_dir, 'trajectory_training_results.txt'), 'w') as f:
        for line in trajectory_results:
            f.write(line + '\n')

    # 保存模型
    torch.save(trajectory_model.state_dict(), os.path.join(output_dir, 'trajectory_best_model.pth'))

    # 输出比较结果
    print("\nComparison of Test Accuracies:")
    print(f"Scatter plot model test accuracy: {scatter_test_acc:.4f}")
    print(f"Trajectory plot model test accuracy: {trajectory_test_acc:.4f}")

# 使用示例：比较散点图和轨迹图的训练效果
folder = "../../IQ_signal_plots"  # 图像存储的文件夹
compare_scatter_trajectory_training(folder, batch_size=32, epochs=20, lr=0.001, limit_per_class=1000)


Using device: cuda:0
Loading scatter plot images...
Total classes: 150
Class 0: 1000 images
Class 1: 1000 images
Class 2: 1000 images
Class 3: 1000 images
Class 4: 1000 images
Class 5: 1000 images
Class 6: 1000 images
Class 7: 1000 images
Class 8: 1000 images
Class 9: 1000 images
Class 10: 1000 images
Class 11: 1000 images
Class 12: 1000 images
Class 13: 1000 images
Class 14: 1000 images
Class 15: 1000 images
Class 16: 1000 images
Class 17: 1000 images
Class 18: 1000 images
Class 19: 1000 images
Class 20: 1000 images
Class 21: 1000 images
Class 22: 1000 images
Class 23: 1000 images
Class 24: 1000 images
Class 25: 1000 images
Class 26: 1000 images
Class 27: 1000 images
Class 28: 1000 images
Class 29: 1000 images
Class 30: 1000 images
Class 31: 1000 images
Class 32: 1000 images
Class 33: 1000 images
Class 34: 1000 images
Class 35: 1000 images
Class 36: 1000 images
Class 37: 1000 images
Class 38: 1000 images
Class 39: 1000 images
Class 40: 1000 images
Class 41: 1000 images
Class 42: 1000 

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\ZY/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:07<00:00, 6.30MB/s]
Epoch 1/20 - Training: 100%|██████████| 3750/3750 [12:33<00:00,  4.98it/s]


Epoch [1/20], Train Loss: 4.4250, Train Acc: 0.0548, Val Loss: 4.1462, Val Acc: 0.0969


Epoch 2/20 - Training: 100%|██████████| 3750/3750 [11:14<00:00,  5.56it/s]


Epoch [2/20], Train Loss: 3.2561, Train Acc: 0.2816, Val Loss: 2.7957, Val Acc: 0.3824


...

Epoch [18/20], Train Loss: 0.1444, Train Acc: 0.9562, Val Loss: 4.5325, Val Acc: 0.5211


Epoch 19/20 - Training: 100%|██████████| 3750/3750 [13:31<00:00,  4.62it/s]


Epoch [19/20], Train Loss: 0.1383, Train Acc: 0.9577, Val Loss: 4.5938, Val Acc: 0.5207


Epoch 20/20 - Training: 100%|██████████| 3750/3750 [12:49<00:00,  4.87it/s]


Epoch [20/20], Train Loss: 0.1259, Train Acc: 0.9617, Val Loss: 4.5153, Val Acc: 0.5214
Best Validation Accuracy: 0.5266
Test Accuracy: 0.5214

Comparison of Test Accuracies:
Scatter plot model test accuracy: 0.5126
Trajectory plot model test accuracy: 0.5214
